In [ ]:
import requests
import sqlite3
import pandas as pd

# NOAA API token
api_token = 'rDZYpxCxcNrIUzeserZMcYDaaHCNPcFZ'

# connect to the DB
conn = sqlite3.connect('C:/Users/Mark Rozenberg/Foobal-Climate/Data/Main_DB.db')

In [ ]:
conn.execute('''
CREATE TABLE IF NOT EXISTS countries (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    alpha_2 TEXT NOT NULL
)
''')
conn.commit()

In [ ]:
import pycountry
for country in pycountry.countries:
    conn.execute('''
    INSERT INTO countries (name, alpha_2) VALUES (?, ?)
    ''', (country.name, country.alpha_2))